<a href="https://colab.research.google.com/github/yudintsev-sergey/Pandas_task/blob/main/Load_grib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Монтируем диск на My Drive

In [2]:
from google.colab import drive
drive.mount("/ShardDrives/")

Mounted at /ShardDrives/


In [3]:
import os
import sys

Определяем рабочую директорию проекта

In [4]:
path_to_module = '/ShardDrives/MyDrive/Colab Notebooks/Meteo'
sys.path.append(path_to_module)
os.chdir('/ShardDrives/MyDrive/Colab Notebooks/Meteo/')

Импортируем необходимые библиотеки и запускаем отсчет времени, которое алгоритм потратит на выполнение программы

In [5]:
import math as ma
import datetime as dt
from datetime import timedelta
import pytz
import numpy as np
import pandas as pd
import urllib.request
from urllib.error import URLError

import time
start = time.time()
end = time.time()
print("Время выполнения фрагмента программы = ", (end-start), " сек")



Время выполнения фрагмента программы =  2.5987625122070312e-05  сек


In [6]:
%%capture
!pip install pygrib
import pygrib
print('Done!')

In [7]:
# @title Создаем форму для ввода данных
# @markdown ### Широта ОП:
op_lat = 70.1 # @param {type:"number"}
# @markdown ### Долгота ОП:
op_lon = 39.3 # @param {type:"number"}
# @markdown ### Широта точки цели:
tgt_lat = 70.25 # @param {type:"number"}
# @markdown ### Долгота точки цели:
tgt_lon = 39.25 # @param {type:"number"}

In [8]:
lat_min = str(ma.floor(op_lat / 0.25) * 0.25)
lon_min = str(ma.floor(op_lon / 0.25) * 0.25)
lat_max = str(ma.ceil(op_lat / 0.25) * 0.25)
lon_max = str(ma.ceil(op_lon / 0.25) * 0.25)
print(lat_min, lon_min, lat_max, lon_max)


70.0 39.25 70.25 39.5


Формируем дату в формате ГГГГММДД

In [9]:
d = ''
tz_utc = pytz.timezone("UTC")
utc_now = dt.datetime.now(tz_utc)
current_hour = int(utc_now.strftime("%H"))
date = utc_now.strftime("%Y%m%d")
print(date)

20231212


Подключаем локальные функции из файла myfunc.py

In [36]:
# Функция выбора доступного по времени файла
def dir_hour(ch: int):
    global d
    if ch.__floordiv__(6) == 0:
        d = '/00/atmos&file=gdas.t00z.pgrb2.0p25.'
    elif ch.__floordiv__(6) == 1:
        d = '/06/atmos&file=gdas.t06z.pgrb2.0p25.'
    elif ch.__floordiv__(6) == 2:
        d = '/12/atmos&file=gdas.t12z.pgrb2.0p25.'      #12 change 0
    elif ch.__floordiv__(6) == 3:
        d = '/18/atmos&file=gdas.t18z.pgrb2.0p25.'
    else:
        pass
    return d
#from myfunc import dir_hour
#from my_func import dir_hour
#a = H_t(240, 98000, 100000)
#print(a)
##b = dir_hour(current_hour)
#print(b)

Формируем адрес для скачки файла *.grib2

In [37]:
thttp = 'https://nomads.ncep.noaa.gov/cgi-bin/filter_gdas_0p25.pl?dir=/gdas.'
t_dir = date + dir_hour(current_hour) + 'f009'
t_ptuv = '&var_PRMSL=on&var_TMP=on&var_UGRD=on&var_VGRD=on'
t_level = '&lev_mean_sea_level=on&lev_1000_mb=on&lev_975_mb=on&lev_950_mb=on&lev_925_mb=on&lev_900_mb=on&lev_850_mb=on&lev_800_mb=on&lev_750_mb=on&lev_700_mb=on&lev_650_mb=on&lev_600_mb=on&lev_550_mb=on&lev_500_mb=on'
t_region = '&subregion=&toplat=' + lat_max + '&leftlon=' + lon_min + '&rightlon=' + lon_max + '&bottomlat=' + lat_min

url = thttp + t_dir + t_ptuv + t_level + t_region
print(url)


https://nomads.ncep.noaa.gov/cgi-bin/filter_gdas_0p25.pl?dir=/gdas.20231212/06/atmos&file=gdas.t06z.pgrb2.0p25.f009&var_PRMSL=on&var_TMP=on&var_UGRD=on&var_VGRD=on&lev_mean_sea_level=on&lev_1000_mb=on&lev_975_mb=on&lev_950_mb=on&lev_925_mb=on&lev_900_mb=on&lev_850_mb=on&lev_800_mb=on&lev_750_mb=on&lev_700_mb=on&lev_650_mb=on&lev_600_mb=on&lev_550_mb=on&lev_500_mb=on&subregion=&toplat=70.25&leftlon=39.25&rightlon=39.5&bottomlat=70.0


Проверяем наличие запрошенной версии. Если ее нет, сдвигаемся на 6 часов и скачиваем

In [38]:
req = urllib.request.Request(url)

try:
    urllib.request.urlopen(req)
except urllib.error.URLError as e:
    if e.reason != 'Not Found':
        print('check the http')
    elif e.reason == 'Not Found':
        print('minus 06')
        current_hour = current_hour - 6
        t_dir = date + dir_hour(current_hour) + 'f009'
        url = thttp + t_dir + t_ptuv + t_level + t_region
        print(url)
    else:
        pass
# urllib.request.urlretrieve(url, 'tmp.grib2')
file, headers = urllib.request.urlretrieve(url)
grbs = pygrib.open(file)

Структура файла *.grib2 (открыт файл как grbs) - демо по широтам и долготам (ячейку можно не исполнять, если структура файла понятна, это просто демо)

In [ ]:
import pprint
print('Широты ячейки = ')
pprint.pprint(grbs[1].latlons()[0])
print(grbs[1].latlons()[0][0][0], grbs[1].latlons()[0][0][1], grbs[1].latlons()[0][1][0], grbs[1].latlons()[0][1][1])
print('Долготы ячейки = ')
pprint.pprint(grbs[1].latlons()[1])
print(grbs[1].latlons()[1][0][0], grbs[1].latlons()[1][0][1], grbs[1].latlons()[1][1][0], grbs[1].latlons()[1][1][1])

Структура файла *.grib2 (открыт файл как grbs) - демо по температуре, широтному и меридиональному ветру (тоже можно не исполнять, если структура файла понятна)

In [40]:
i = 2          # i/P: 1/sea_level, 2/50000Pa, 5/55000, 8/60000, 11/65000, ..., 35/97500, 38/100000
print(grbs[i])
print(grbs[i + 1])
print(grbs[i + 2])
pprint.pprint(grbs[i].values)
print(grbs[i].values[0][0], grbs[i].values[0][1], grbs[i].values[1][0], grbs[i].values[1][1], )
pprint.pprint(grbs[i + 1].values)
pprint.pprint(grbs[i + 2].values)

2:Temperature:K (instant):regular_ll:isobaricInhPa:level 50000 Pa:fcst time 9 hrs:from 202312120600
3:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 50000 Pa:fcst time 9 hrs:from 202312120600
4:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 50000 Pa:fcst time 9 hrs:from 202312120600
array([[239.44171875, 239.42171875],
       [239.33171875, 239.30171875]])
239.44171875 239.42171875 239.33171875 239.30171875
array([[ 8.45200806,  8.45200806],
       [10.85200806, 10.85200806]])
array([[-4.62028809, -4.72028809],
       [-4.62028809, -4.82028809]])


Реальный счет массива

In [ ]:
msg = grbs[1]
coord_vals = msg.latlons()                         # Печатаем заголовок, хотя нужен ли он?
print("[hp]", \
      "[", coord_vals[0][0, 0], "/", coord_vals[1][0, 0], "]", \
      "[", coord_vals[0][0, 1], "/", coord_vals[1][0, 1], "]", \
      "[", coord_vals[0][1, 0], "/", coord_vals[1][1, 0], "]", \
      "[", coord_vals[0][1, 1], "/", coord_vals[1][1, 1], "]")

print(grbs[1].values)                     # Давление на уровне моря в вершинах координатной ячейки

Читаем температурные кортежи-четверки (в углах координатной ячейки)

In [ ]:
t_klv = np.zeros(52).reshape((13, 2, 2))

for i in range(1, 14):
    t_klv[[i - 1]] = grbs[41 - 3 * i].values   # Температура (К) от высоты Р = 100.000 и вверх до Р =50.000
pprint.pprint(t_klv)
print(t_klv[0][0][0])                          # Угловые элементы, просто проверка
print(t_klv[12][1][1])

Читаем стандартную линейку давлений (100.000 - 50.000) и кортеж давлений на уровне моря в координатной ячейке

In [ ]:
p_lin = np.zeros(13)
for i in range(1, 14):
    p: int = str(grbs[3*i-1]).find('Pa:fcst')
    p_lin[14 - i - 1] = float(str(grbs[3*i-1])[p-7:p-1])                  # индекс в обратном порядке, так удобнее
pprint.pprint(p_lin)

p_sea_level = grbs[1].values                                              # Давление на уровне моря в координатной ячейке
print(p_sea_level)

Пересчет давлений в высоты (Pa ==> metres)

In [ ]:
p_div_p0 = np.zeros(52).reshape((13, 2, 2))
for i in range(13):
    p_div_p0[i] = p_lin[i] / p_sea_level

h_tp = np.zeros(52).reshape((13, 2, 2))
h_tp = -29.21 * t_klv * np.log(p_div_p0)

pprint.pprint(h_tp)
print(h_tp[0][0][0])                          # Угловые элементы, просто проверка
print(h_tp[12][1][1])

Читаем ветровые (широтные) кортежи-четверки (в углах координатной ячейки)

In [ ]:
u_lat =np.zeros(52).reshape((13, 2, 2))

for i in range(1, 14):
    u_lat[[i - 1]] = grbs[42 - 3 * i].values   # Широтная составляющая ветра от высоты Р = 100.000 и вверх до Р =50.000
pprint.pprint(u_lat)
print(u_lat[0][0][0])                          # Угловые элементы, просто проверка
print(u_lat[12][1][1])

Читаем ветровые (меридиональные) кортежи-четверки (в углах координатной ячейки)

In [ ]:
v_lon =np.zeros(52).reshape((13, 2, 2))

for i in range(1, 14):
    v_lon[[i - 1]] = grbs[43 - 3 * i].values   # Меридиональная составляющая ветра от высоты Р = 100.000 и вверх до Р =50.000
pprint.pprint(v_lon)
print(v_lon[0][0][0])                          # Угловые элементы, просто проверка
print(v_lon[12][1][1])